### Probability Graph Model Benchmark
Whithin this notebook we will compare technics to evaluate a probability graph model for a given seed article through out its neighbors.

In [105]:
from wikipydia import dict_storage, wikidb, parse, wikisyn
WikisynDict = wikisyn.WikisynDict
from collections import Counter, defaultdict
from tabulate import tabulate
import re
from ThreadPool import ThreadPool
import sys

In [2]:
wiki_db = wikidb.WikiDb()

In [3]:
def print_data(headers, data_list):
    """Function to print data tabulated """
    return print(tabulate(data_list, headers=headers))

In [4]:
last_percent_reported = None
def download_progress_hook(count, totalSize):
  """A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 5% change in download progress.
  """
  global last_percent_reported
  percent = int(count * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent

In [5]:
#Function to download bunch of wikipedia pages at once if they are not present
check_and_download__done = 0
def check_and_download(pages):
    global check_and_download__done
    check_and_download__done = 0
    
    n_tasks = len(pages)
    
    print("Tasks to go: " + str(n_tasks))
    
    # Function to be executed in a thread
    def download_stuff(page):
        global check_and_download__done
        #try:
        wiki_db.get_article_by_href(page)
        #except:
            #print("Failed to get page " + page + ". Timed out.")
        #finally:
        check_and_download__done += 1
        download_progress_hook(check_and_download__done, n_tasks)
            #percentage = round(check_and_download__done * 100 / n_tasks)
            #if percentage % 5 == 0:
                #print("{0}%.....".format(percentage), end="")
            #print("Done " + str(check_and_download__done) + "/" + str(n_tasks))

    # Instantiate a thread pool with 5 worker threads
    pool = ThreadPool(50)

    pool.map(download_stuff, pages)
    pool.wait_completion()
    
    print("\nFinishing downloading. Done tasks: " + str(check_and_download__done) + "/" + str(n_tasks))

In [32]:
def get_links_score(href):
    synDict = WikisynDict()
    
    wikiart, _ = wiki_db.get_article_by_href(href)
    links = wikiart.links()
    page_text = wikiart.text()
    
    #Populate syndict
    for href, text in links:
        synDict.add_link(href, text)
        
    #Get matches
    links_score = Counter()
        
    for link_href in synDict.keys():
        for l_text, l_score in synDict[link_href]:
            matches = re.findall('[^a-zA-Z0-9_]' + re.escape(l_text) + '[^a-zA-Z0-9_]', page_text, re.IGNORECASE)
            matches_score = len(matches) * l_score
            links_score[link_href] += matches_score

    scores_sum = sum(links_score.values())
            
    #norm_links_scores = list(map(lambda a: (a[0],synDict[a[0]], a[1]/scores_sum), links_score.most_common()))
    norm_links_scores = list(map(lambda a: (a[0], a[1]/scores_sum), links_score.most_common()))
    #norm_links_scores = list(map(lambda a: (a[0],list(map(lambda b: b[0], synDict[a[0]])), a[1]/scores_sum), links_score.most_common()))
    
    return norm_links_scores

In [52]:
#get_links_score("JavaScript")

In [43]:
def get_references_score(page_href, top=0):
    """Function to calculate each article reference score."""
    
    page_links_scores = sorted(get_links_score(page_href), key=lambda a: a[1], reverse=True)
    
    if top > 0:
        page_links_scores = page_links_scores[:top]
    
    #Ensure everything has been downloaded first    
    hrefsToDownload = [link_href for link_href, _ in page_links_scores]        
    check_and_download(hrefsToDownload)
    
    references_scores = Counter()
    
    for link_href, score in page_links_scores:
        
        link_article, _ = wiki_db.get_article_by_href(link_href)
        link_title = link_article.title()
        
        #If there is already a title already place, sum the scores
        references_scores[link_title] += score
        
    #Ensure everything sums to one
    score_sum = sum(references_scores.values())    
    norm_score_edges = [(ref, score/score_sum) for ref, score in references_scores.most_common()]
        
    return norm_score_edges

In [51]:
#get_references_score("MQTT", 20)

In [73]:
def get_href_edges(page_href, top):
    page_article, _ = wiki_db.get_article_by_href(page_href)
    page_title = page_article.title()
    
    edges = list()
    loop_back = False
    for link_title, score in get_references_score(page_href, top):
        #Skip loop back edges
        if page_title == link_title:
            loop_back = True
            continue
            
        edges.append(((page_title, link_title), score))
        
    #Ensures everything sums to one if a loop back occurs
    if loop_back:
        score_sum = sum([score for edge, score in edges])
        edges = [(edge, score/score_sum) for edge, score in edges]
        
    return edges

In [74]:
#get_href_edges("Service-oriented_architecture", 20)

In [78]:
from urllib.parse import quote

def get_graph_edges(seed_href, deepness, top=-1):
    """Get all the edges by tranversing the graph by t_deep starting on seed_href."""
    
    #Ensure transversal deepness is max of 3
    assert deepness <= 3
    
    seed_article, _ = wiki_db.get_article_by_href(seed_href)
    seed_title = seed_article.title()
    
    done_titles = list()
    todo_queue = list()
    
    todo_queue.append(seed_title)
    
    edges = list()
    
    for i in range(deepness):
        print("Working on batch {0} of {1}".format(i+1, deepness))
        
        current_todo_queue = todo_queue
        todo_queue = list()
        
        print("queue size: {0}".format(len(current_todo_queue)))
        
        #while len(current_todo_queue) > 0:
            #next_title = current_todo_queue.pop() 
        for next_title in current_todo_queue:
            
            if next_title in done_titles:
                continue
        
            #edges_scores = get_node_edges_scores(quote(next_title), top)
            edges_scores = get_href_edges(quote(next_title), top)
        
            done_titles.append(next_title)
            
            #Save new edges and next to do titles
            for edge, score in edges_scores:
                edges.append((edge, score))
                
                if not edge[1] in done_titles:
                    todo_queue.append(edge[1])
    
    return edges           

In [147]:
#get_graph_edges("c%2b%2b", 1, 20)

In [80]:
import networkx as nx

def get_graph(edges):
    #Create a directed graph
    graph = nx.DiGraph()
    for edge in edges:
        graph.add_edge(edge[0], edge[1])
    return graph

In [81]:
def extract_path_tuples(path):
    """Function that generates all path tuples from a list path."""
    path_tuples = []
    for i, _ in enumerate(path):
        if i == 0:
            continue
        path_tuples.append((path[i-1], path[i]))

    return path_tuples

def get_features(graph, seed_node, cutoff, deep_rank_scores=None):
    """
    Function to compute the prereq probabilities for every node.
    Compute deeprank and bidirection rank. TO BE EXPLAINED

    Returns: bidir_probs, deeprank_probs, n_paths, min_depths, max_depths

    """

    #Compute number of edges per node
    nodes_edges = dict()
    for n1, n2 in graph.edges():
        #init nodes dict if not initiated
        if not n1 in nodes_edges:
            nodes_edges[n1] = 0
        nodes_edges[n1] += 1

    #Compute initial edges probabilities
    #For bidirection rank we set 1 in case the edge is unidirectional and 0.5 in case bidirectional.
    #For deeprank we compute the fraction of the edge over all the edges in the same node.

    ### Later we need to create other methods to compute proper distribution for the cases above. ### 

    bidir_edges_values = dict()
    deeprank_edges_values = dict()

    #edges_values = dict()
    for n1, n2 in graph.edges():
        if deep_rank_scores == None:
            deeprank_edges_values[(n1, n2)] = 1.0 / nodes_edges[n1]
        
        if graph.has_edge(n2, n1):
            bidir_edges_values[(n1, n2)] = 0.5
        else:
            bidir_edges_values[(n1, n2)] = 1

    if deep_rank_scores != None:
        deeprank_edges_values = dict(deep_rank_scores)

    #Now compute all the paths to the target seed_node and sequence probabilities to each path
    bidir_probs = dict() #Probabilities of reach seed_node from each node based on bidir values
    deeprank_probs = dict() #Probabilities of reach seed_node from each node based on deeprank values
    min_depths = dict() #Each node min depth
    max_depths = dict() #Each node max depth
    ns_paths = dict() #Each node number of paths

    # create dicts for every feature extracted
    # try get insights from kmeans
    # try to find something to deploy FAST (the energy applied must be low!)

    n_nodes = nx.number_of_nodes(graph)

    #Iterate thru all the graph nodes
    for i, node in enumerate(graph.nodes()):

        print("Working on node {0}/{1}".format(i+1,n_nodes))

        #Init min max depth
        min_depth = cutoff + 2
        max_depth = 0

        #Skip seed_node since we do not want verify paths to itself
        if node == seed_node:
            continue

        n_paths = 0
        total_bidir_prob = 0
        total_deeprank_prob = 0

        for path in nx.all_simple_paths(graph, source=seed_node, target=node, cutoff=cutoff):

            n_paths += 1
            partial_bidir_prob = 1.0
            partial_deeprank_prob = 1.0

            #Computes min-max depth
            max_depth = max(max_depth, len(path))
            min_depth = min(min_depth, len(path))

            #Iterate the path tuples
            for i, edge_tuple in enumerate(extract_path_tuples(path)):
                partial_bidir_prob *= bidir_edges_values[edge_tuple]
                partial_deeprank_prob *= deeprank_edges_values[edge_tuple]

            total_bidir_prob += partial_bidir_prob
            total_deeprank_prob += partial_deeprank_prob

        #After processing all node paths, save the final values    
        bidir_probs[node] = total_bidir_prob / n_paths
        deeprank_probs[node] = total_deeprank_prob
        min_depths[node] = min_depth
        max_depths[node] = max_depth
        ns_paths[node] = n_paths 

    return bidir_probs, deeprank_probs, ns_paths, min_depths, max_depths

In [185]:

class WikiGraph(nx.DiGraph):
    """A enhanced directed graph to calculate stuff based on wikipedia links."""
        
    def _extract_path_tuples(self, path):
        """Function that generates all path tuples from a list path."""
        path_tuples = []
        for i, _ in enumerate(path):
            if i == 0:
                continue
            path_tuples.append((path[i-1], path[i]))

        return path_tuples
    
    def pagerank(self):
        return nx.pagerank(self)
    
    def number_of_paths(self, seed_node, cutoff):
        number_of_paths = Counter()
        
        for node in self.nodes():
            for path in nx.all_simple_paths(self, source=seed_node, target=node, cutoff=cutoff):
                number_of_paths[node] += 1
                #all_paths[node].append((self._extract_path_tuples(path)))
                
        return number_of_paths
        
    def deeprank(self, seed_node, cutoff, skiplevels=0):
        """
        Function to compute the prereq probabilities for every node.
        Compute deeprank and bidirection rank. TO BE EXPLAINED

        Returns: deeprank_probs, n_paths, min_depths, max_depths
        """
        
        assert self.has_node(seed_node)
        
        #Get all paths from seed_node to every other node on the graph
        all_paths = defaultdict(list)
        for node in self.nodes():
            #Skip seed_node since we do not want verify paths to itself
            if node == seed_node:
                continue
            for path in nx.all_simple_paths(self, source=seed_node, target=node, cutoff=cutoff):
                all_paths[node].append((self._extract_path_tuples(path)))
        
        #Compute deeprank
        deeprank = Counter()
        this_graph = self
        for node, node_paths in all_paths.items(): #Iterate thru all target nodes
            for path in node_paths: #Iterate thru all paths to the target node
                if len(path) <= skiplevels:
                    continue
                
                partial_deeprank = 1.0
                for i, (source, target) in enumerate(path):#Iterate thru all edges in the path                    
                    partial_deeprank *= this_graph[source][target]["weight"] 
                deeprank[node] += partial_deeprank
        
        return deeprank

In [189]:
#Get graph edges
href = "JavaScript"
deep = 3
top = 10

target_title = wiki_db.get_article_by_href(href)[0].title()
graph_edges = get_graph_edges(href, deep, top)
wiki_db.save()

Working on batch 1 of 3
queue size: 1
Tasks to go: 10

Finishing downloading. Done tasks: 10/10
Working on batch 2 of 3
queue size: 10
Tasks to go: 10

Finishing downloading. Done tasks: 10/10
Tasks to go: 10

Finishing downloading. Done tasks: 10/10
Tasks to go: 10

Finishing downloading. Done tasks: 10/10
Tasks to go: 10

Finishing downloading. Done tasks: 10/10
Tasks to go: 10

Finishing downloading. Done tasks: 10/10
Tasks to go: 10

Finishing downloading. Done tasks: 10/10
Tasks to go: 10

Finishing downloading. Done tasks: 10/10
Tasks to go: 10

Finishing downloading. Done tasks: 10/10
Tasks to go: 10

Finishing downloading. Done tasks: 10/10
Tasks to go: 10

Finishing downloading. Done tasks: 10/10
Working on batch 3 of 3
queue size: 91
Tasks to go: 10

Finishing downloading. Done tasks: 10/10
Tasks to go: 10

Finishing downloading. Done tasks: 10/10
Tasks to go: 10

Finishing downloading. Done tasks: 10/10
Tasks to go: 10

Finishing downloading. Done tasks: 10/10
Tasks to go: 1

In [190]:
#Construct graph
wiki_graph = WikiGraph()

for edge, score in graph_edges:
    wiki_graph.add_edge(edge[0], edge[1], weight=score)
    
#Calculate stuff

deeprank = wiki_graph.deeprank(target_title, 4)
in_degree = wiki_graph.in_degree()
pagerank = wiki_graph.pagerank()
number_of_paths = wiki_graph.number_of_paths(target_title, 4)

#Multiply things
deep_in_rank = dict()
for key in deeprank.keys():
    deep_in_rank[key] = deeprank[key] * in_degree[key]
    
print_data(['Article', 'Score'], sorted(deep_in_rank.items(), key=lambda a:a[1], reverse=True))

#print("DEEP RANK")
#print_data(['Article', 'Score'], sorted(.items(), key=lambda a:a[1], reverse=True))
#print("\n\nIN DEGREE")
#print_data(['Article', 'Score'], sorted().items(), key=lambda a:a[1], reverse=True))
#print("\n\nPAGE RANK")
#print_data(['Article', 'Score'], sorted(.items(), key=lambda a:a[1], reverse=True))
#print("\n\nNUMBER OF PATHS")
#print_data(['Article', 'Score'], sorted(.items(), key=lambda a:a[1], reverse=True))

Implement own algorithm to find paths in order to give attention to some top nodes
save everything to excel to multiply things
check lessons on ensemble for some insight how to combine them
check reinforcement course


#Later implement importance stuff based on sections
# try get insights from kmeans
# try to find something to deploy FAST (the energy applied must be low!)


Article                                               Score
----------------------------------------------  -----------
Microsoft                                       2.59831
Object (computer science)                       1.69087
HTML                                            1.50833
Subroutine                                      1.4804
Java (programming language)                     0.816647
Class (computer programming)                    0.69919
Google                                          0.688766
Computer program                                0.59928
C (programming language)                        0.449392
Microsoft Outlook                               0.323522
Computer                                        0.24287
Software                                        0.240331
World Wide Web Consortium                       0.198787
Operating system                                0.19374
Programming language                            0.19053
Prototype JavaScript Framework     

In [82]:
target_article = "Convolutional neural network"

scored_edges = get_graph_edges(target_article, 2, 10)
edges = [edge for edge,score in scored_edges]

#article_title = get_wiki_article_by_href(target_article).title
art,_ = wiki_db.get_article_by_href(target_article)
article_title = art.title()

#print(scored_edges)
#print(edges)
graph = get_graph(edges)

bidir_probs, deeprank_probs, ns_paths, min_depths, max_depths = get_features(graph, article_title, 4, scored_edges)

print_data(["Title", "Bidir", "Deep"])


print("\n")
for title, dr in sorted(deeprank_probs.items(), key=lambda a: a[1], reverse=True):
    print("{0}        {1}        {2}        {3}        {4}        {5}" \
          .format(title, round(dr,3), round(bidir_probs[title],3), ns_paths[title], min_depths[title], max_depths[title]))
    #print(title, round(dr,3))
    
#edges_score = get_graph_edges("Computer_vision",2, 20)
#wiki_db.save()
#print_data(["Link", "Score"], sorted(edges_score, key=lambda a: a[1], reverse=True))


Working on batch 1 of 2
queue size: 1
Tasks to go: 10
10%20%30%50%60%40%80%70%90%
Finishing downloading. Done tasks: 10/10
Working on batch 2 of 2
queue size: 9
Tasks to go: 10
10%30%20%40%50%60%70%80%90%100%
Finishing downloading. Done tasks: 10/10
Tasks to go: 10
10%20%30%40%50%60%70%80%90%100%
Finishing downloading. Done tasks: 10/10
Tasks to go: 10
10%20%30%50%40%60%70%80%90%100%
Finishing downloading. Done tasks: 10/10
Tasks to go: 10
20%10%30%40%50%60%70%80%90%100%
Finishing downloading. Done tasks: 10/10
Tasks to go: 3
..100%
Finishing downloading. Done tasks: 3/3
Tasks to go: 10
10%20%30%40%50%60%70%80%90%100%
Finishing downloading. Done tasks: 10/10
Tasks to go: 10
10%20%40%60%30%50%70%80%90%
Finishing downloading. Done tasks: 10/10
Tasks to go: 10
10%20%30%40%50%60%70%80%90%100%
Finishing downloading. Done tasks: 10/10
Tasks to go: 10
10%20%30%40%50%60%70%80%90%100%
Finishing downloading. Done tasks: 10/10
Working on node 1/84
Working on node 2/84
Working on node 3/84
Working

TypeError: print_data() missing 1 required positional argument: 'data_list'